In [43]:
import numpy as np
data_name = 'NYCTaxi' #'NYCBike1' 'NYCBike2'

In [44]:

data_train = np.load(f'./{data_name}/train.npz')
for file in data_train.files:
    print(file, data_train[file].shape)
data_train = np.load(f'./{data_name}/train_full.npz')
for file in data_train.files:
    print(file, data_train[file].shape)

x (1912, 35, 200, 2)
y (1912, 1, 200, 2)
x_offsets (35, 1)
y_offsets (1, 1)
x (1912, 35, 200, 4)
y (1912, 1, 200, 4)
x_offsets (35, 1)
y_offsets (1, 1)


In [45]:

data_val = np.load(f'./{data_name}/val.npz')
for file in data_val.files:
    print(file, data_val[file].shape)
data_val = np.load(f'./{data_name}/val_full.npz')
for file in data_val.files:
    print(file, data_val[file].shape)
# print(data_val['x'][0,:,0,:])
# print(data_val['y'][0,:,0,:])

x (274, 35, 200, 2)
y (274, 1, 200, 2)
x_offsets (35, 1)
y_offsets (1, 1)
x (274, 35, 200, 4)
y (274, 1, 200, 4)
x_offsets (35, 1)
y_offsets (1, 1)


In [46]:
data_test = np.load(f'./{data_name}/test.npz')
for file in data_test.files:
    print(file, data_test[file].shape)
data_test = np.load(f'./{data_name}/test_full.npz')
for file in data_test.files:
    print(file, data_test[file].shape)

x (546, 35, 200, 2)
y (546, 1, 200, 2)
x_offsets (35, 1)
y_offsets (1, 1)
x (546, 35, 200, 4)
y (546, 1, 200, 4)
x_offsets (35, 1)
y_offsets (1, 1)


In [47]:

data_adj = np.load(f'./{data_name}/adj_mx.npz')
for file in data_adj.files:
    print(file, data_adj[file].shape)
    print(data_adj[file])
# Save to graph.txt
# 对于NYCBike1(16*8), graph的node顺序按照h16w8的gird顺序
np.savetxt('graph.txt', data_adj['adj_mx'], fmt='%d')

adj_mx (200, 200)
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
